In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools, os, torch

from sim import Scheduler, Random, Leitner
from data_process import process_original, reduce_df, eval_thresh, reduce_lexemes
from get_trajectory import get_traj

%load_ext autoreload
%autoreload 2

In [ ]:
!cd data && ls

## Load data (clean if necesarry)

In [ ]:
if not os.path.exists("data/cleaned.csv"):
    process_original()
    

df = pd.read_csv("data/cleaned.csv")

## Data Exploration

In [4]:
lang_map = {'de' : 0, 'en': 1, 'es': 2, 'fr': 3, 'it': 4, 'pt': 5}
l_map = pd.read_csv("data/lexeme_map.csv")

df.groupby('learning_language').count().loc[:, 'user_id']

learning_language
0    1452597
1    5014791
2    3407689
3    1873734
4     793935
5     311480
Name: user_id, dtype: int64

We have about 5 million english items, 3 million Spanish 1.9 million French and 1.4 million German. Italian and Portugese each have hundreds of thousands. It would be useful to restrict out studies to just the English users so we reduce the dimensionality of our action and state spaces.

There are 43.8 thousand learners (trajectories) we have to provide our RL agents.



In [5]:
df = df.loc[df['learning_language'] == 1].copy()
df = df.drop(['learning_language'], axis=1)
reduce_df(df)
df.loc[:, 'difficulty'] = df.loc[:, 'difficulty'].astype(np.float32)
df.loc[:, 'difficulty'] -= df.loc[:, 'difficulty'].mean()
df.loc[:, 'difficulty'] /= df.loc[:, 'difficulty'].std()

In [6]:
english_counts = df.groupby('lexeme_id').count().loc[:, 'timestamp']
n_lex = len(english_counts)
print(f"There are {n_lex} lexemes")

There are 2983 lexemes


In [8]:
n_items = int(1000)

eval_thresh(df, english_counts, n_items)

For threshold 1000 there are 15.32% lexemes above and 84.68% below

There would be 90.48% of data included and 9.52% of data excluded


In [9]:
df, included = reduce_lexemes(df, n_items)

In [12]:
states, actions, itl, lti = get_traj(df, included)

## Run Behavioral Cloning

In [10]:
import torch.nn as nn
import torch
import random
import cs285.infrastructure.pytorch_util as ptu
from tqdm import tqdm

In [11]:
traj_idxs = list(states.keys())
def sample_traj(states, actions, idxs):
    first_idx = idxs.pop()
    states_cat = states[first_idx]
    actions_cat = actions[first_idx]
    while states_cat.shape[0] < batch_size and len(idxs) > 0:
        new_idx = idxs.pop()
        states_cat = np.concatenate((states_cat, states[new_idx]))
        actions_cat = np.concatenate((actions_cat, actions[new_idx]))
    return states_cat, actions_cat

NameError: name 'states' is not defined